# Data Generation using Modelling and Simulation for Machine Learning

## Simulation Tool: SimPy (Discrete-Event Simulation)

We simulate a **multi-server queuing system** (e.g., a bank/hospital service center) where:
- Customers arrive at random intervals
- They wait in a queue if all servers are busy
- They get served and leave

We vary the simulation parameters randomly, run 1000 simulations, and use the generated data to train and compare ML models.

## Step 1 & 2: Install and Import SimPy

In [ ]:
!pip install simpy numpy pandas scikit-learn xgboost matplotlib seaborn -q

In [ ]:
import simpy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

## Step 3: Define Simulation Parameters and Bounds

| Parameter | Description | Lower Bound | Upper Bound |
|-----------|-------------|-------------|-------------|
| `arrival_rate` | Mean customer arrival rate (customers/unit time) | 0.5 | 5.0 |
| `service_rate` | Mean service rate per server (customers/unit time) | 0.3 | 3.0 |
| `num_servers` | Number of parallel servers | 1 | 10 |
| `queue_capacity` | Maximum queue length | 5 | 100 |
| `simulation_time` | Total simulation duration | 200 | 1000 |

In [ ]:
PARAM_BOUNDS = {
    'arrival_rate': (0.5, 5.0),
    'service_rate': (0.3, 3.0),
    'num_servers': (1, 10),
    'queue_capacity': (5, 100),
    'simulation_time': (200, 1000)
}

## Step 4: Build the Queuing Simulation

In [ ]:
def customer(env, name, server, service_rate, stats):
    """A customer arrives, waits for a server, gets served, and leaves."""
    arrival_time = env.now
    with server.request() as req:
        result = yield req | env.timeout(0)  # check if queue is available
        if req in result:
            wait_start = env.now
            yield req
            wait_time = env.now - arrival_time
            stats['wait_times'].append(wait_time)

            service_time = np.random.exponential(1.0 / service_rate)
            yield env.timeout(service_time)
            stats['served'] += 1
        else:
            stats['lost'] += 1


def customer_process(env, server, arrival_rate, service_rate, queue_capacity, stats):
    """A customer arrives, waits for a server, gets served, and leaves."""
    arrival_time = env.now

    if len(server.queue) >= queue_capacity:
        stats['lost'] += 1
        return

    with server.request() as req:
        yield req
        wait_time = env.now - arrival_time
        stats['wait_times'].append(wait_time)

        service_time = np.random.exponential(1.0 / service_rate)
        yield env.timeout(service_time)
        stats['served'] += 1


def arrival_generator(env, server, arrival_rate, service_rate, queue_capacity, stats):
    """Generate customers at random intervals."""
    customer_id = 0
    while True:
        inter_arrival = np.random.exponential(1.0 / arrival_rate)
        yield env.timeout(inter_arrival)
        customer_id += 1
        stats['total_arrivals'] += 1
        env.process(customer_process(env, server, arrival_rate, service_rate, queue_capacity, stats))


def monitor_queue(env, server, stats, interval=1.0):
    """Periodically record queue length."""
    while True:
        stats['queue_lengths'].append(len(server.queue))
        yield env.timeout(interval)


def run_simulation(arrival_rate, service_rate, num_servers, queue_capacity, simulation_time):
    """Run a single queuing simulation and return metrics."""
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=num_servers)

    stats = {
        'wait_times': [],
        'queue_lengths': [],
        'served': 0,
        'lost': 0,
        'total_arrivals': 0
    }

    env.process(arrival_generator(env, server, arrival_rate, service_rate, queue_capacity, stats))
    env.process(monitor_queue(env, server, stats))
    env.run(until=simulation_time)

    avg_wait_time = np.mean(stats['wait_times']) if stats['wait_times'] else 0
    avg_queue_length = np.mean(stats['queue_lengths']) if stats['queue_lengths'] else 0
    server_utilization = (stats['served'] / (num_servers * simulation_time * service_rate)) if service_rate > 0 else 0
    server_utilization = min(server_utilization, 1.0)
    loss_rate = stats['lost'] / stats['total_arrivals'] if stats['total_arrivals'] > 0 else 0

    return {
        'avg_wait_time': avg_wait_time,
        'avg_queue_length': avg_queue_length,
        'server_utilization': server_utilization,
        'customers_served': stats['served'],
        'customers_lost': stats['lost'],
        'loss_rate': loss_rate
    }

## Step 5: Generate 1000 Simulations

In [ ]:
NUM_SIMULATIONS = 1000

records = []

for i in range(NUM_SIMULATIONS):
    arrival_rate = np.random.uniform(*PARAM_BOUNDS['arrival_rate'])
    service_rate = np.random.uniform(*PARAM_BOUNDS['service_rate'])
    num_servers = np.random.randint(*PARAM_BOUNDS['num_servers'])
    queue_capacity = np.random.randint(*PARAM_BOUNDS['queue_capacity'])
    simulation_time = np.random.randint(*PARAM_BOUNDS['simulation_time'])

    results = run_simulation(arrival_rate, service_rate, num_servers, queue_capacity, simulation_time)

    record = {
        'arrival_rate': arrival_rate,
        'service_rate': service_rate,
        'num_servers': num_servers,
        'queue_capacity': queue_capacity,
        'simulation_time': simulation_time,
        **results
    }
    records.append(record)

    if (i + 1) % 100 == 0:
        print(f"Completed {i + 1}/{NUM_SIMULATIONS} simulations")

df = pd.DataFrame(records)
print(f"\nDataset shape: {df.shape}")
df.head()

In [ ]:
df.describe()

In [ ]:
df.to_csv('simulation_data.csv', index=False)
print("Dataset saved to simulation_data.csv")

## Data Exploration

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
output_cols = ['avg_wait_time', 'avg_queue_length', 'server_utilization',
               'customers_served', 'customers_lost', 'loss_rate']

for ax, col in zip(axes.flatten(), output_cols):
    ax.hist(df[col], bins=30, edgecolor='black', alpha=0.7)
    ax.set_title(f'Distribution of {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('output_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.savefig('correlation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## Step 6: Train and Compare ML Models

**Target variable:** `avg_wait_time`

**Features:** `arrival_rate`, `service_rate`, `num_servers`, `queue_capacity`, `simulation_time`

In [ ]:
feature_cols = ['arrival_rate', 'service_rate', 'num_servers', 'queue_capacity', 'simulation_time']
target_col = 'avg_wait_time'

X = df[feature_cols]
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(kernel='rbf'),
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
    'MLP Neural Network': MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
}

results = []

for name, model in models.items():
    if name in ['SVR', 'KNN', 'MLP Neural Network']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({
        'Model': name,
        'MSE': round(mse, 4),
        'RMSE': round(rmse, 4),
        'MAE': round(mae, 4),
        'R2 Score': round(r2, 4)
    })
    print(f"{name:25s} | RMSE: {rmse:.4f} | MAE: {mae:.4f} | R2: {r2:.4f}")

results_df = pd.DataFrame(results).sort_values('R2 Score', ascending=False)
results_df.to_csv('model_comparison.csv', index=False)
print("\nResults saved to model_comparison.csv")

In [ ]:
print("\n" + "="*70)
print("MODEL COMPARISON TABLE")
print("="*70)
print(results_df.to_string(index=False))
print("="*70)
best = results_df.iloc[0]
print(f"\nBest Model: {best['Model']} (R2 = {best['R2 Score']}, RMSE = {best['RMSE']})")

## Result Visualizations

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# R2 Score comparison
axes[0].barh(results_df['Model'], results_df['R2 Score'], color='steelblue', edgecolor='black')
axes[0].set_xlabel('R2 Score')
axes[0].set_title('Model Comparison - R2 Score')
axes[0].axvline(x=0, color='red', linestyle='--', alpha=0.5)

# RMSE comparison
axes[1].barh(results_df['Model'], results_df['RMSE'], color='coral', edgecolor='black')
axes[1].set_xlabel('RMSE')
axes[1].set_title('Model Comparison - RMSE (lower is better)')

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
metrics = ['MSE', 'RMSE', 'MAE', 'R2 Score']
colors = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6']

for ax, metric, color in zip(axes.flatten(), metrics, colors):
    bars = ax.barh(results_df['Model'], results_df[metric], color=color, edgecolor='black', alpha=0.8)
    ax.set_xlabel(metric)
    ax.set_title(f'Model Comparison - {metric}')
    for bar, val in zip(bars, results_df[metric]):
        ax.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
                f' {val:.4f}', va='center', fontsize=8)

plt.tight_layout()
plt.savefig('all_metrics_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Predicted vs Actual for the best model
best_model_name = results_df.iloc[0]['Model']
best_model = models[best_model_name]

if best_model_name in ['SVR', 'KNN', 'MLP Neural Network']:
    y_pred_best = best_model.predict(X_test_scaled)
else:
    y_pred_best = best_model.predict(X_test)

plt.figure(figsize=(8, 8))
plt.scatter(y_test, y_pred_best, alpha=0.5, edgecolors='black', linewidth=0.5)
min_val = min(y_test.min(), y_pred_best.min())
max_val = max(y_test.max(), y_pred_best.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Actual Avg Wait Time')
plt.ylabel('Predicted Avg Wait Time')
plt.title(f'Predicted vs Actual - {best_model_name}')
plt.legend()
plt.savefig('predicted_vs_actual.png', dpi=150, bbox_inches='tight')
plt.show()

## Feature Importance (Best Tree-Based Model)

In [ ]:
# Use Random Forest for feature importance
rf_model = models['Random Forest']
importances = rf_model.feature_importances_

feat_imp_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': importances
}).sort_values('Importance', ascending=True)

plt.figure(figsize=(8, 5))
plt.barh(feat_imp_df['Feature'], feat_imp_df['Importance'], color='teal', edgecolor='black')
plt.xlabel('Feature Importance')
plt.title('Random Forest - Feature Importance')
plt.savefig('feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

## Conclusion

The results table above shows the comparison of 10 ML models on the simulation-generated dataset. The best model is identified based on the highest R2 score and lowest RMSE. See the README for a detailed discussion of results.